<a href="https://colab.research.google.com/github/liorhirsch/Kaggle-Landmark-ML-Challange/blob/master/Custom%20NN/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuring the environment either to local or remote usage

In [0]:
BASE_PATH='drive/My Drive'
# BASE_PATH="E:/Projects/ML_Project/Landmark_Kaggle_Challange"
MODEL_NAME = "1.0"

TRAINING_DATA_DIR = BASE_PATH + '/photos'
VALIDATION_DATA_DIR = BASE_PATH + '/validation'
TEST_DATA_PATH = BASE_PATH + '/test'

MODEL_ENDING = ".H5"

RESULTS_PATH=BASE_PATH + "/results/"
SAVING_DIR_PATH=BASE_PATH + "/models/"
FULL_SAVING_PATH = SAVING_DIR_PATH + MODEL_NAME + MODEL_ENDING

# Upload data from google drive

In [10]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Build training data


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from joblib import Parallel, delayed
import multiprocessing

**Augmentated Data Generation**

Creates images generators both for the test and the training data, to create additional data

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen=ImageDataGenerator(
                              rotation_range=0,
                              width_shift_range=0,
                               height_shift_range=0,
                               shear_range=0,
                               zoom_range=[0.8, 1.25],
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='reflect',
                               data_format='channels_last',
                               brightness_range=[0.5, 1.5]) #included in our dependencies

test_datagen = ImageDataGenerator(
                               rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.01,
                               zoom_range=[0.8, 1.25],
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='reflect',
                               data_format='channels_last',
                               brightness_range=[0.5, 1.5]) #included in our dependencies


 Uses the data as a stream object to enable batching on it instead of loading them all together to the ram using the ImageDataGenerator

In [33]:
image_size = (100,100)
batch_size = 25

train_generator=train_datagen.flow_from_directory(TRAINING_DATA_DIR,
                                                 target_size=image_size,
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

validation_generator = test_datagen.flow_from_directory(
                                        VALIDATION_DATA_DIR,
                                         target_size=image_size,
                                         color_mode='rgb',
                                        class_mode = "categorical")

Found 67559 images belonging to 100 classes.
Found 7458 images belonging to 100 classes.


# Validate the data

In [0]:
x_batch, y_batch = next(train_generator)
# print(x_batch) # RGB 2d array
# print(y_batch) # The label (class)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20, 20))
for i in range(batch_size):
    plt.subplot(5,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image = x_batch[i]
    image= image.astype(np.uint8)
    plt.imshow(image)

# Build The Network

In [0]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv3D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

**Normalize the images.**

In [0]:
x_batch = x_batch/255.0
# print(x_batch)

**Build the NN**

In [0]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = x_batch.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatening the data
model.add(Flatten())
model.add(Dense(150))
model.add(Activation("relu"))

#output layer
model.add(Dense(100))
model.add(Activation('softmax'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        18464     
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16928)             0         
__________

We convert the generator's output from binary array that contains 1 in the related class, to array contains the indexes of the classes for each picture in the batch

In [22]:
# y=[a.tolist().index(1) for a in y_batch];
# print(y)

[32, 5, 12, 32, 6]


# Train

Let us configure a loss function and optimizer.

The generator gives us in y a vector contains match percentages for each label, so we want to change it to show 0 or 1 using caregorical_crossentropy

In [0]:
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

we calculate the amount of batches in an ephoch (For fitting the model)

In [31]:
step_size_train=train_generator.n//train_generator.batch_size
print(step_size_train)

13511


Fix for the error : 
OSError: image file is truncated (14 bytes not processed)

In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True 

#https://stackoverflow.com/questions/12984426/python-pil-ioerror-image-file-truncated-with-big-images

We define an early stopper to avoid wasting compute resources and time

In [0]:
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=1, mode='auto')

In [0]:
import time

networkfileName = RESULTS_PATH + "/" + MODEL_NAME + "{}" + MODEL_ENDING.format(int(time.time()))
                      
checkpoint = ModelCheckpoint(networkfileName, monitor='val_acc', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

Now we train the model with fit_generator instead of regular fit.

In [0]:
history = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train, # step=how much batches in one epoc
                   validation_data = validation_generator,
                   epochs=5
                   ,callbacks = [checkpoint, early])

Epoch 1/5
   17/13511 [..............................] - ETA: 40:52:03 - loss: 15.4734 - acc: 0.0400

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


  543/13511 [>.............................] - ETA: 33:11:42 - loss: 15.7322 - acc: 0.0239

# Examine the results

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Save the model

Create results directory if it doesn't exist

In [0]:
import os
if not os.path.exists(SAVING_DIR_PATH):
    os.makedirs(SAVING_DIR_PATH)

In [0]:
model.save(FULL_SAVING_PATH)  # creates a HDF5 file 'my_model.h5'

# Load the model from h5 file

load the model from the saved file

In [0]:
from tensorflow.keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model(FULL_SAVING_PATH)

In [0]:
train_loss, train_acc = model.evaluate(X,y)

In [0]:
print("loss: ",train_loss, "acc: ",  train_acc)

# Test the model on the TestSet

Lets evaluate our model on the test set :


In [0]:
train_loss, train_acc = model.evaluate(x_batch,y)

In [0]:
test_datagen=ImageDataGenerator()
test_generator=test_datagen.flow_from_directory(TEST_DATA_PATH,
                                                  target_size=image_size,
                                                  color_mode='rgb',
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

step_size_test=test_generator.n//test_generator.batch_size

test_loss, test_acc = model_final.evaluate_generator(generator=test_generator, 
                               steps=step_size_test)
                               


In [0]:
print(train_loss)
print(train_acc)